In [2]:
pip install music21 --user

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import music21 as m21
import json
import numpy as np
import setuptools.dist
import tensorflow.keras as keras
from fractions import Fraction

In [2]:
DATASET_PATH = ".\\MusicXml"
TRANSPOSE_PATH = ".\\Transpose"
PREPROCESS_PATH = ".\\Preprocess"
MINIMUM_DURATION = 1/6
SEQUENCE_LENGTH = 48

In [3]:
ACCEPTABLE_DURATIONS = [
    Fraction(4),
    Fraction(3),
    Fraction(2),
    Fraction(3, 2),
    Fraction(1),
    Fraction(3, 4),
    Fraction(2, 3),
    Fraction(1, 2),
    Fraction(1, 3),
    Fraction(1, 4),
    Fraction(1, 6)
]

In [4]:
def load_songs(dataset_path):
    songs = []
    for filename in os.listdir(dataset_path):
        filepath = os.path.join(dataset_path, filename)
        if os.path.isfile(filepath) and filename.endswith(".xml"):
            song = m21.converter.parse(filepath)
            songs.append(song)

    return songs

In [5]:
def filter_songs_by_duration(song):
    for note in song.flatten().notesAndRests:
        duration = note.duration.quarterLength
        if Fraction(duration).limit_denominator() not in ACCEPTABLE_DURATIONS and note.duration.quarterLength != 0:
            print(note.duration.quarterLength)
            return False
    return True

In [6]:
def generate_all_key_transpositions(song):
    transposed_songs = []
    for i in range(0, 12):
        transposed_song = song.transpose(i)
        transposed_songs.append(transposed_song)
    return transposed_songs

In [5]:
def transpose_to_C_or_A(song):
    key = song.analyze("key")
    if key.mode == "major":
        interval = m21.interval.Interval(key.tonic, m21.pitch.Pitch("C"))
    elif key.mode == "minor":       
        interval = m21.interval.Interval(key.tonic, m21.pitch.Pitch("A"))
    # print(interval)
    transposed_song = song.transpose(interval)
    return transposed_song

In [27]:
def get_minimum_duration(songs):
    minimum_duration = 1
    for song in songs:
        for note in song.flat.notesAndRests and note.duration.quarterLength != 0:
            if note.duration.quarterLength != 0 and note.duration.quarterLength < minimum_duration:
                minimum_duration = note.duration.quarterLength
    return minimum_duration

In [6]:
def encode_solo(song, minimum_duration):
    encoded_solo = []
    
    for element in song.flatten().notesAndRests:
        if isinstance(element, m21.note.Note): symbol = element.pitch.midi
        elif isinstance(element, m21.note.Rest): symbol = "r"
        duration = element.duration.quarterLength / minimum_duration
        for i in range(int(duration)):
            if i == 0: encoded_solo.append(symbol)
            else: encoded_solo.append("_")

    encoded_solo = " ".join(map(str, encoded_solo))
    return encoded_solo

In [7]:
def encode_chords(song, minimum_duration):
    encoded_chords = []
    current_chord = None
    current_duration = 0

    for element in song.flatten():
        if isinstance(element, m21.harmony.ChordSymbol):
            if current_chord: 
                for i in range(int(current_duration)):
                    if i == 0: encoded_chords.append(current_chord.figure)
                    else: encoded_chords.append("_")
            current_chord = element
            current_duration = 0
        elif isinstance(element, m21.note.Note) or isinstance(element, m21.note.Rest):
            current_duration += element.duration.quarterLength / minimum_duration

    if current_chord:
        for i in range(int(current_duration)):
            if i == 0: encoded_chords.append(current_chord.figure)
            else: encoded_chords.append("_")
    encoded_chords = " ".join(map(str, encoded_chords))
    return encoded_chords

In [8]:
songs = load_songs(DATASET_PATH)
print(songs[0].metadata.title)
# minimum_duration = get_minimum_duration(songs)

Another Hairdo


In [9]:
filtered_songs = []

for song in songs:
    if filter_songs_by_duration(song):
        filtered_songs.append(song) 

print(len(filtered_songs))

4/5
0.125
0.125
0.125
1/12
0.125
0.125
1/5
1/5
0.125
1/12
0.125
1/5
1/5
0.125
0.125
0.125
0.125
1/5
0.125
0.125
0.125
28


In [11]:
all_key_songs = []
for song in songs:
    all_key_songs.extend(generate_all_key_transpositions(song))

In [18]:
def save_transcribed_songs(song, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for idx, song in enumerate(songs):
        for i in range(0, 12):
            transposed_song = song.transpose(i)
            filename = os.path.join(output_dir, f"score_{idx}_transposed_{i}.xml")
            transposed_song.write("musicxml", fp = filename)

In [19]:
save_transcribed_songs(songs, TRANSPOSE_PATH)

In [12]:
print(all_key_songs[11].metadata.title)
print(encode_solo(all_key_songs[11], minimum_duration))
print(encode_chords(all_key_songs[11], minimum_duration))

Another Hairdo
r _ _ _ _ _ 69 _ _ _ _ _ 74 _ 76 _ 74 _ 69 _ _ _ _ _ 72 _ _ _ _ _ 73 _ _ _ _ _ r _ _ _ _ _ 69 _ _ _ _ _ 74 _ 76 _ 74 _ 69 _ _ _ _ _ 72 _ _ _ _ _ 73 _ _ _ _ _ r _ _ _ _ _ 69 _ _ _ _ _ 74 _ 76 _ 74 _ 69 _ _ _ _ _ 72 _ _ _ _ _ 73 _ _ _ _ _ r _ _ _ _ _ 69 _ _ _ _ _ 71 _ _ _ _ _ 67 _ _ _ _ _ 67 _ _ _ _ _ _ _ _ _ _ _ r _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 84 _ _ 83 _ _ 81 _ _ 79 _ _ 78 _ _ 76 _ _ 74 _ _ 68 _ _ 69 _ _ 71 _ _ 72 _ _ 69 _ _ 71 _ _ _ _ _ _ _ _ _ _ _ 71 _ _ _ _ _ _ _ _ _ _ _ r _ _ _ _ _ _ _ _ _ _ _ r _ _ _ _ _ 69 _ _ _ _ _ 73 _ _ _ _ _ 76 _ _ _ _ _ 78 _ _ _ _ _ 81 _ _ _ _ _ 83 _ _ _ _ _ 80 _ _ _ _ _ 81 _ _ 83 _ _ 81 _ _ 80 _ _ 78 _ _ _ _ _ 77 _ _ _ _ _ 76 _ _ _ _ _ 74 _ _ _ _ _ 73 _ _ _ _ _ 71 _ _ _ _ _ 70 _ _ _ _ _ 79 _ _ _ _ _ 78 _ 79 _ 78 _ 76 _ _ _ _ _ 74 _ _ _ _ _ 66 _ _ _ _ _ 69 _ _ _ _ _ 73 _ _ _ _ _ 71 _ _ _ _ _ _ _ _ _ _ _ r _ _ _ _ _ 84 _ _ _ _ _ 84 _ _ _ _ _ 81 _ _ _ _ _ _ _ _ _ _ _ 79 _ _ _ _ _ 76

In [11]:
all_songs_in_C = []
for song in songs:
    all_songs_in_C.append(transpose_to_C_or_A(song))

In [12]:
print(all_songs_in_C[11].metadata.title)
print(encode_solo(all_songs_in_C[11], minimum_duration))
print(encode_chords(all_songs_in_C[11], minimum_duration))

Blue Bird
64 _ _ _ _ _ 55 _ _ _ _ _ 57 _ _ _ 55 _ _ _ 57 _ _ _ 57 _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 55 _ _ _ _ _ 58 _ _ _ _ _ 57 _ _ _ _ _ 58 _ _ _ _ _ 57 _ _ _ _ _ 57 _ _ _ _ _ 56 _ _ _ _ _ _ _ _ _ _ _ 64 _ _ _ _ _ 67 _ _ _ _ _ _ _ _ _ _ _ 64 _ _ 62 _ _ 59 _ _ 55 _ _ 58 _ _ _ _ _ 66 _ _ _ _ _ 66 _ _ _ _ _ _ _ _ _ _ _ r _ _ _ _ _ 68 _ _ 69 _ _ 72 _ _ 69 _ _ 67 _ _ 65 _ _ 69 _ _ 65 _ _ 64 _ _ 62 _ _ 68 _ _ 67 _ _ r _ _ _ _ _ 64 _ _ _ _ _ 64 _ _ _ _ _ 60 _ _ _ 55 _ _ _ 60 _ _ _ 64 _ _ _ _ _ 64 _ _ _ _ _ 60 _ _ _ 55 _ _ _ 60 _ _ _ 64 _ _ _ _ _ 64 _ _ _ _ _ r _ _ _ _ _ _ _ _ _ _ _ r _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 64 _ _ _ _ _ 64 _ _ _ _ _ 60 _ _ _ 55 _ _ _ 60 _ _ _ 64 _ _ _ _ _ 67 _ _ _ _ _ 69 _ _ _ _ _ 70 _ _ _ _ _ 70 _ _ _ _ _ _ _ _ _ _ _ r _ _ _ _ _ _ _ _ _ _ _ r _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 63 _ _ _ _ _ 63 _ _ _ _ _ 60 _ _ _ 55 _ _ _ 60 _ _ _ 63 _ _ _ _ _ 63 _ _ _ _ _ 60 _ _ _ 55 _ _ _ 60 _ _ _ 63 _ _ _ _ _ 63 _ _ _ _ _ r _ _ _ _ _ _ _ _ _ _ _ r _ _ _ _ 

In [40]:
def get_chord_set(songs):
    list = []
    for song in songs:
        for element in song.flatten():
            if isinstance(element, m21.harmony.ChordSymbol):
                list.append(element.figure)
    return sorted(set(list))

In [61]:
print(get_chord_set(songs))
# print(len(get_chord_set(all_songs_in_C)))
print(len(get_chord_set(songs)))

['A', 'A-', 'A-7', 'A-dim', 'A-m', 'A7', 'Am', 'Aø7', 'B', 'B-', 'B-7', 'B-m', 'B7', 'Bdim', 'Bm', 'C', 'C#m', 'C-', 'C-7', 'C-m', 'C/G', 'C6', 'C7', 'Cm', 'Cø7', 'D-', 'D-7', 'D-m', 'D7', 'Dm', 'Dø7', 'E-', 'E-7', 'E-dim', 'E-m', 'E7', 'Edim', 'Em', 'Eø7', 'F', 'F#7', 'F#ø7', 'F7', 'Fdim', 'Fm', 'G', 'G-7', 'G-dim', 'G7', 'Gm', 'Gø7']
51


In [10]:
def create_single_string(songs, sequence_length):
    new_song_delimiter = "/ " * sequence_length
    combined_solos = ""
    combined_chords = ""

    for song in songs:
        solo = encode_solo(song, MINIMUM_DURATION)
        chords = encode_chords(song, MINIMUM_DURATION)
        combined_solos += solo + " " + new_song_delimiter
        combined_chords += chords + " " + new_song_delimiter

    return combined_solos[:-1], combined_chords[:-1]

In [11]:
solos, chords = create_single_string(filtered_songs, SEQUENCE_LENGTH)
with open(".\\solos.txt", "w") as fp:
    fp.write(solos)

with open(".\\chords.txt", "w") as fp:
    fp.write(chords)

In [12]:
print(len(solos))
print(len(chords))

115323
106516


In [15]:
def create_mapping(song_string):
    mappings = {}

    chars = song_string.split()
    vocabulary = list(set(chars))

    for idx, symbol in enumerate(vocabulary):
        mappings[symbol] = idx

    return mappings, len(vocabulary)

In [16]:
lookup_table, VOCABULARY_SIZE = create_mapping(solos + chords)
with open(".\\lookup_table.json", "w") as fp:
    json.dump(lookup_table, fp, indent=4)

In [13]:
def convert_to_int(solos, chords, lookup_table):
    int_solos = []
    int_chords = []

    solo_chars = solos.split()
    for symbol in solo_chars:
        int_solos.append(lookup_table[symbol])

    chords_chars = chords.split()
    for symbol in chords_chars:
        int_chords.append(lookup_table[symbol])

    return int_solos, int_chords
    

In [17]:
int_solos, int_chords = convert_to_int(solos, chords, lookup_table)

print(len(int_solos))
print(len(int_chords))

51140
51672


In [18]:
def generate_training_sequences(int_solos, int_chords, sequence_length):
    inputs =  []
    targets = [] 

    num_sequences = len(int_solos) - sequence_length
    for i in range(num_sequences):
        input_sequence = int_chords[i : i + sequence_length + 1] + int_solos[i : i + sequence_length]
        # print(len(input_sequence))
        target = int_solos[i + sequence_length]
        inputs.append(input_sequence)
        targets.append(target)

    # vocabulary_size = len(set(int_solos))
    inputs = keras.preprocessing.sequence.pad_sequences(inputs, maxlen = sequence_length + 1, padding = "pre")
    inputs = keras.utils.to_categorical(inputs, num_classes = VOCABULARY_SIZE)
    targets = np.array(targets)

    return inputs, targets

In [19]:
inputs, targets = generate_training_sequences(int_solos, int_chords, SEQUENCE_LENGTH)

In [38]:
OUTPUT_UNITS = VOCABULARY_SIZE
NUM_UNITS = [256]
LOSS = "sparse_categorical_crossentropy"
LEARNING_RATE = 0.001
EPOCHS = 50
BATCH_SIZE = 64

In [39]:
def build_model(output_units = OUTPUT_UNITS, num_units = NUM_UNITS, loss = LOSS, learning_rate = LEARNING_RATE):
    input = keras.layers.Input(shape = (None, output_units))
    x = keras.layers.LSTM(num_units[0])(input)
    x = keras.layers.Dropout(0.2)(x)

    output = keras.layers.Dense(output_units, activation = "softmax")(x)
    
    model = keras.Model(input, output)
    model.compile(loss = loss, 
                  optimizer = keras.optimizers.Adam(learning_rate = learning_rate),
                  metrics = ["accuracy"])

    model.summary()

    return model

In [40]:
model = build_model(OUTPUT_UNITS, NUM_UNITS, LOSS, LEARNING_RATE)

Model: "functional_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, None, 83)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ (None, 256)            │       348,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 83)             │        21,331 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 369,491 (1.41 MB)

 Trainable params: 369,491 (1.41 MB)

 Non-trainable params: 0 (0.00 B)

In [41]:
model.fit(inputs, targets, epochs = EPOCHS, batch_size = BATCH_SIZE)

Epoch 1/50
799/799 ━━━━━━━━━━━━━━━━━━━━ 129s 157ms/step - accuracy: 0.6962 - loss: 1.6152
Epoch 2/50
799/799 ━━━━━━━━━━━━━━━━━━━━ 118s 147ms/step - accuracy: 0.7122 - loss: 1.1715
Epoch 3/50
799/799 ━━━━━━━━━━━━━━━━━━━━ 117s 146ms/step - accuracy: 0.7221 - loss: 1.0299
Epoch 4/50
799/799 ━━━━━━━━━━━━━━━━━━━━ 139s 142ms/step - accuracy: 0.7340 - loss: 0.9719
Epoch 5/50
799/799 ━━━━━━━━━━━━━━━━━━━━ 120s 150ms/step - accuracy: 0.7447 - loss: 0.9140
Epoch 6/50
799/799 ━━━━━━━━━━━━━━━━━━━━ 120s 150ms/step - accuracy: 0.7511 - loss: 0.8771
Epoch 7/50
799/799 ━━━━━━━━━━━━━━━━━━━━ 115s 144ms/step - accuracy: 0.7582 - loss: 0.8544
Epoch 8/50
799/799 ━━━━━━━━━━━━━━━━━━━━ 115s 144ms/step - accuracy: 0.7683 - loss: 0.8199
Epoch 9/50
799/799 ━━━━━━━━━━━━━━━━━━━━ 143s 145ms/step - accuracy: 0.7669 - loss: 0.8189
Epoch 10/50
799/799 ━━━━━━━━━━━━━━━━━━━━ 119s 148ms/step - accuracy: 0.7744 - loss: 0.7848
Epoch 11/50
799/799 ━━━━━━━━━━━━━━━━━━━━ 117s 146ms/step - accuracy: 0.7771 - loss: 0.7714
Epoch 12

In [43]:
model.save("model.keras")

In [14]:
def chord_frequency(songs):
    freq_dict  = {}
    for song in songs:
        for element in song.flatten():
            if isinstance(element, m21.harmony.ChordSymbol):
                chord = element.figure
                if chord in freq_dict.keys(): freq_dict[chord] += 1
                else: freq_dict[chord] = 1
    return freq_dict


In [20]:
def key_from_chords(song):
    chord_symbols_stream = m21.stream.Stream()
    for element in song.flatten():
        if isinstance(element, m21.harmony.ChordSymbol):
            chord_symbols_stream.append(element)
    return chord_symbols_stream.analyze("key")


In [22]:
print(key_from_chords(songs[0]))
print(songs[0].analyze("key"))

b minor
B- major


In [20]:
def sample_with_temperature(probabilities, temperature):
    predictions = np.log(probabilities)/temperature
    probabilities = np.exp(predictions)/np.sum(np.exp(predictions))

    choices = range(len(probabilities))
    index = np.random.choice(choices, p = probabilities)
    
    return index

In [21]:
def generate_improvised_solo(solo_seed, chords, num_steps, max_sequence_length, temperature, lookup_table, model):
    # solo_seed = solo_seed.split()
    # chords = chords.split()

    solo = solo_seed
    # print(solo)
    solo_seed = ["/" for _ in range(SEQUENCE_LENGTH)] + solo_seed
    solo_seed = [lookup_table[symbol] for symbol in solo_seed]

    chords = ["/" for _ in range(SEQUENCE_LENGTH)] + chords
    chords = [lookup_table[symbol] for symbol in chords]

    for i in range(num_steps):
        solo_seed = solo_seed[-max_sequence_length:]
        chord_input = chords[i : i + SEQUENCE_LENGTH + 1]
        seed = chord_input + solo_seed
        
        onehot_seed = keras.utils.to_categorical(seed, num_classes = len(lookup_table))
        onehot_seed = onehot_seed[np.newaxis, ...]

        probabilities = model.predict(onehot_seed)[0]
        output = sample_with_temperature(probabilities, temperature)
        solo_seed.append(output)
        
        output_symbol = [k for k, v in lookup_table.items() if v == output][0]
        if output_symbol == "/": break
        # print(output_symbol)
        solo.append(output_symbol)

    return solo

In [23]:
test_model = keras.models.load_model("model.keras")

C:\Users\hridd\AppData\Roaming\Python\Python312\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 7 variables whereas the saved optimizer has 12 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [24]:
test_song = filtered_songs[0]
print(test_song.metadata.title)
test_seed = encode_solo(test_song, MINIMUM_DURATION)
print(len(test_seed))
test_chords = encode_chords(test_song, MINIMUM_DURATION)
print(test_chords)
print(len(test_chords.split()))

Another Hairdo
3272
B-7 _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ B-7 _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ B-7 _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ B-7 _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ E-7 _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ E-7 _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ B-7 _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ B-7 _ _ _ _ _ _ _ _ _ _ _ G7 _ _ _ _ _ _ _ _ _ _ _ Cm _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ F7 _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ B-7 _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ B-7 _ _ _ _ _ _ _ _ _ _ _ F7 _ _ _ _ _ _ _ _ _ _ _ B-7 _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ B-7 _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ B-7 _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ B-7 _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ E-7 _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ E-7 _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ B-7 _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ G7 _ _ _ _ _ _ _ _ _ _ _ _ _ _

In [43]:
test_song = filtered_songs[-1]
with open("lookup_table.json", "r") as fp:
    test_mapping = json.load(fp)
print(test_song.metadata.title)
test_seed = encode_solo(test_song, MINIMUM_DURATION).split()[:12]
print(test_seed)
test_chords = encode_chords(test_song, MINIMUM_DURATION).split()
print(test_chords)
test_steps = len(test_chords) - SEQUENCE_LENGTH + 36
print(test_steps)

Yardbird Suite
['r', '_', '_', '72', '_', '_', '_', '_', '_', '_', '_', '_']
['C', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', 'Fm', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', 'B-7', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', 'C7', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', 'B-7', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', 'A7', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', 'D7', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', 'G7', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', 'Em', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', 'A7', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', 'Dm', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', 'G7', '_',

In [44]:
generated_solo = generate_improvised_solo(test_seed, test_chords, test_steps, SEQUENCE_LENGTH, 0.3, test_mapping, test_model)
print(generated_solo)
print(len(generated_solo))
print(len(test_chords))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 546ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1

In [59]:
def save_solo(solo, minimum_duration, format = "musicxml", file_name = "generated.musicxml"):
    stream = m21.stream.Stream()
    current_note = None
    current_duration = 1

    for note in solo:
        if note != "_":
            if current_note:
                duration = current_duration * minimum_duration
                if current_note == "r": m21_event = m21.note.Rest(quarterLength = duration)
                else: m21_event = m21.note.Note(int(current_note), quarterLength = duration)
                stream.append(m21_event)
                current_duration = 1
            current_note = note
        else:
            current_duration += 1
            
    if current_note:
        duration = current_duration * minimum_duration
        if current_note == "r": m21_event = m21.note.Rest(quarterLength = duration)
        else: m21_event = m21.note.Note(int(current_note), quarterLength = duration)
        stream.append(m21_event)

    stream.write(format, file_name)  
    return stream 
    

In [60]:
saved_solo = save_solo(generated_solo, MINIMUM_DURATION)

In [57]:
def save_lead_sheet(chords, solo, minimum_duration, file_name="lead_sheet.musicxml"):
    score = m21.stream.Score()

    chord_part = m21.stream.Part()
    solo_part = m21.stream.Part()

    current_chord = None
    current_chord_duration = 0
    current_solo_duration = 0

    for chord, note in zip(chords, solo):
        if current_chord != chord:
            if current_chord is not None:
                chord_measure = m21.stream.Measure()
                chord_measure.append(m21.harmony.ChordSymbol(current_chord))
                chord_part.append(chord_measure)
            current_chord = chord
            current_chord_duration = 0

        if note != 'r':  # Handle pitch notes and held notes
            if current_solo_duration > 0:
                solo_measure = m21.stream.Measure()
                if current_solo_duration > 1:  # Add held notes/rests
                    solo_measure.append(m21.note.Rest(quarterLength=minimum_duration * current_solo_duration))
                else:  # Add single note/rest
                    solo_measure.append(m21.note.Note(note, quarterLength=minimum_duration))
                solo_part.append(solo_measure)
                current_solo_duration = 0

            note_duration = int(note) * minimum_duration
            solo_measure = m21.stream.Measure()
            solo_measure.append(m21.note.Note(note, quarterLength=note_duration))
            solo_part.append(solo_measure)
        else:  # Handle rests
            current_solo_duration += 1

        current_chord_duration += 1

        if current_chord_duration == 24:  # End of measure
            current_chord = None
            current_chord_duration = 0

    # Handle any remaining rests at the end of the solo
    if current_solo_duration > 0:
        solo_measure = m21.stream.Measure()
        solo_measure.append(m21.note.Rest(quarterLength=minimum_duration * current_solo_duration))
        solo_part.append(solo_measure)

    score.insert(0, chord_part)
    score.insert(0, solo_part)

    score.write('musicxml', file_name)

In [58]:
output_stream = save_lead_sheet(test_chords, generated_solo, MINIMUM_DURATION)

PitchException: Cannot make a step out of '_'